## HW2 - LP formulations, duals, and a first ILP

In this second homework assignment we will focus on a few more LP formulations, a duality question, and a first ILP. In contrast to the first homework, we'll provide a little bit less skeleton code, so you may want to rely on code samples you already saw in class (e.g., notebooks shared on Canvas, HW1 solutions, etc.) to help you with the basic skeleton of things needed, like declaring the model, setting the objective, etc.

In [1]:
import gurobipy as gp
from IPython.display import Image

#### Problem 1: Product mix and profit maximization

A canning company operates two plants (South and Central). Its suppliers provide fresh fruit of three different kinds (peaches, pears, and pineapples). The unit cost of peaches, pears, and pineapples is 110, 100, and 90 USD respectively. The shipping costs per unit are (respectively, for peaches, pears, and pineapples) 30, 20, and 60 USD to the southern plant. To the central plant, the shipping costs are 35, 25, and 40 USD. The southern plant has a capacity of 460 units, and labor costs of 260 USD per unit. The central plant has a capacity of 560 units, and labor costs of 210 USD per unit. Labor costs at both plants are independent of fruit type.

Cans of peaches, pears, and pineapples are sold to distributors at 480, 500, and 520 USD per unit respectively. The distributors provide (and pay for) the shipping from the plants.

Assuming that the company wants to maximize its profit, how much of each fruit should they can at each of the factories? (Note: if you do the bonus part, you don't need to write out the answers to the other parts)

- What decision variables would you use to find the optimal shipments?

One variable per fruit/plan combination: $x_{ij}$ equal to the quantity of fruit $i\in\{\text{peach, pear, pineaple}\}$ provided to plant $j\in\{\text{south, central}\}$.

- How would you encode the objective of this LP?

Let $c_i$ be the cost of each fruit $i$, $s_{ij}$ be the shipping cost of each fruit $i$ to plant $j$, and $l_j$ the labor cost per unit of fruit of plant $j$. Let $r_i$ be the sell value of each fruit $i$.

Total fruit $i$ produced: $\sum_j x_{ij}$.

Total fruit in plant $j$: $\sum_j x_{ij}$.

Then, the objective would be to maximize the total profit:

$$\sum_i r_i\left(\sum_j x_{ij}\right) - \sum_i c_i \left(\sum_j x_{ij}\right) - \sum_j l_j\left(\sum_i x_{ij}\right).$$

Note that since the distributor pays for the shipping, we don't need to incorporate that in the objective.

- What kind of constraints will a LP require to solve this question? First describe the effect each constraint would have in English, then write them out (notice that Jupyter should understand latex, so you can either write them out in mathematical form or write out a description like "sum over all variables of this type be at most that" --- in the latter case, make sure to be as precise as possible!)

There is a capacity constraint on how much fruit can be stored at a plant:

$$\sum_i x_{i,south} \leq 460,\ \ \sum_i x_{i,central} \leq 560$$

Bonus: implement the above in gurobi (not required, but if you struggled with HW1 it would be good practice to write it out from scratch!).

In [2]:
plants = ['South', 'Central']
# We won't save the fruits explicitly, and thus enumerate peach, pear, pineapple, as 0th, 1st, 2nd index
# Also, instead of keeping c_i and r_i separately, we just define r_i-c_i
net_revenues = [370, 400, 430]
shipping_costs = {'South': [30,20,60], 'Central': [35,25,40]}
capacities = {'South': 460, 'Central': 560}
labor_costs = {'South': 260, 'Central': 210}

m = gp.Model("primal")
x = {}
fruits = [0,1,2]
for fruit in fruits:
    for plant in plants:
        x[fruit,plant] = m.addVar(lb=0)
for plant in plants:
    m.addConstr(sum(x[fruit,plant] for fruit in fruits) <= capacities[plant])

labor_cost = 0
for plant in plants: labor_cost += labor_costs[plant] * sum(x[fruit,plant] for fruit in fruits)
shipping_cost = 0
for plant in plants: shipping_cost += sum(shipping_costs[plant][fruit] * x[fruit,plant] for fruit in fruits)
net_revenue = 0
for plant in plants: net_revenue += sum(x[fruit,plant]*net_revenues[fruit] for fruit in fruits)
m.setObjective(net_revenue-labor_cost-shipping_cost, gp.GRB.MAXIMIZE)
m.optimize()
for fruit, plant in x:
    if x[fruit,plant].x >0: print(fruit, plant, x[fruit,plant].x)

Using license file /Users/dfreund/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x2394998a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 6e+02]
Presolve removed 2 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.560000000e+05
1 South 460.0
2 Central 560.0


Note that the solution is somewhat silly/trivial: for each plant, we should only can the fruit that maximizes profit (revenue minus unit cost minus labor cost minus shipping cost), and can as much of that as we have capacity for. Under what kind of constraints would the problem have a *more interesting* solution

#### Problem 2: Taking a dual

In class we discussed how to take the dual LP of a given primal LP. In this problem, we will practice taking duals, and remind ourselves both of how the dual solution relates to sensitivity analysis and of how gurobi can give us the dual solutions.

The primal problem we study is as follows:


$\max x + 3\cdot y + 2\cdot z$

subject to:

$ 0\leq x \leq 10$

$0 \leq y \leq 12$

$3\cdot x+y+5\cdot z\leq 10 $

$x + 3\cdot y\leq 12$

$3\cdot z\geq 6$


Below, I already implemented this LP in Gurobi. Make sure you follow all the steps.

In [3]:
# First, we define a variable that contains the optimization model
m = gp.Model("primal")

# Next, we define the three decision variables with their respective lower and upper bounds
x = m.addVar(lb = 0) # 0<= x
y = m.addVar(lb = 0) # 0<= y
z = m.addVar(lb=-gp.GRB.INFINITY) # z can be anything

# Then, we give bounds on x and y being at most 10, respectively 12
bound_x = m.addConstr(x<=10) 
bound_y = m.addConstr(y<=12) 

# Then, we add the other constraints
c1 = m.addConstr(3*x+y+5*z<=10) # 3x + y + 5z<=10
c2 = m.addConstr(x+3*y<=12) # x + 3y<=12
c3 = m.addConstr(3*z>=6) # 3*z>=6

# Next, we set our objective: maximizing x+3y+2z
m.setObjective(x+3*y+2*z, gp.GRB.MAXIMIZE)

m.optimize()

# Then we print the solution values for each variable
print(x.x,y.x,z.x)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 5 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x512860b2
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.000000000e+00
0.0 0.0 2.0


- Rewrite the LP with (i) all variables non-negative, (ii) all constraints with variables on the LHS <= parameters on the right-hand side (probably easiest to copy-paste from the markdown cell above, and then make changes where necessary)


$\max x + 3\cdot y + 2\cdot z$

subject to:

$x \leq 10$

$y \leq 12$

$3\cdot x+y+5\cdot z\leq 10 $

$x + 3\cdot y\leq 12$

$-z\leq -2$

$x,y,z\geq 0$

- On paper (no need to submit), write out the dual of this LP

- Implement dual of the LP in gurobi, solve it, and verify that strong duality holds, i.e., that the optimal solutions of primal and dual match.

In [4]:
# First, we define a variable that contains the optimization model
m2 = gp.Model("dual")

# Next, we define the three decision variables with their respective lower and upper bounds
a = m2.addVar(lb=0)
b = m2.addVar(lb=0)
c = m2.addVar(lb=0)
d = m2.addVar(lb=0)
e = m2.addVar(lb=0)
    
dc1 = m2.addConstr(a + 3 * c + d >= 1)
dc2 = m2.addConstr(b + c + 3 * d >= 3)
dc2 = m2.addConstr(5 * c - e >= 2)

# Next, we set our objective: maximizing x+3y+2z
m2.setObjective(10 * a + 12 * b + 10 * c + 12 * d - 2 * e, gp.GRB.MINIMIZE)

m2.optimize()

# Then we print the solution values for each variable
print(a.x, b.x, c.x, d.x, e.x)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 5 columns and 8 nonzeros
Model fingerprint: 0x2e54ecb6
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  4.000000000e+00
0.0 0.0 3.0 0.0 13.0


- In class we discussed the idea of shadow prices and sensitivity analysis. In Gurobi, as we saw in class, we can find the dual variable of a constraint c by typing c.pi, so for example:

In [5]:
c1.pi

3.0

Can you give a shadow price explanation of what c1.pi being equal to 3 means? Write out, and solve, a new LP (in Gurobi) that illustrates the shadow price of c1 being 3. Do the same for one of the other constraints (c2 or c3), and explain your findings.

In [6]:
# First, we define a variable that contains the optimization model
m = gp.Model("primal")

# Next, we define the three decision variables with their respective lower and upper bounds
x = m.addVar(lb = 0) # 0<= x
y = m.addVar(lb = 0) # 0<= y
z = m.addVar(lb=-gp.GRB.INFINITY) # z can be anything

# Then, we give bounds on x and y being at most 10, respectively 12
bound_x = m.addConstr(x<=10) 
bound_y = m.addConstr(y<=12) 

# Then, we add the other constraints
c1 = m.addConstr(3*x+y+5*z<=11) # 3x + y + 5z<=10 became 11 on the RHS; the objective goes up by at most 3
c2 = m.addConstr(x+3*y<=12) # x + 3y<=12
c3 = m.addConstr(3*z>=6) # 3*z>=6

# Next, we set our objective: maximizing x+3y+2z
m.setObjective(x+3*y+2*z, gp.GRB.MAXIMIZE)

m.optimize()

# Then we print the solution values for each variable
print(x.x,y.x,z.x)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 5 rows, 3 columns and 8 nonzeros
Model fingerprint: 0xbeef9b92
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.000000000e+00
0.0 1.0 2.0


Findings explained:

The value of the dual price is the _shadow price_ of that constraint. We can see it as the local derivative of the objective with respect to the right hand side constraint. Thus, for every unit of increment on the right hand side constraint, we should see (up to) one unit of increment on the objective. 

Indeed, as we created an LP whose right hand side for the first constraint is 1 unit larger, the objective increased in 3 * 1 units.

#### Problem 3: Minimize transportation costs

Consider the following supply network where the number on each arc indicates the per-unit cost of shipping from one location to another. The supply at the factories is the number of units each factory can provide, the demand next to customers indicate how many units are needed.

In [7]:
# DO NOT RUN THIS CELL, ELSE YOU'LL LOSE THE PICTURE OF THE NETWORK!
Image("warehouse_network.png")

FileNotFoundError: No such file or directory: 'warehouse_network.png'

FileNotFoundError: No such file or directory: 'warehouse_network.png'

<IPython.core.display.Image object>

Our goal here is to satisfy all the demand while respecting the supply constraints at the factories, and minimizing the total cost of shipping.

- What decision variables would you use to find the optimal shipments

Let $$A=\{(1,4), (1,5), (1,6), (2,6), (2,7), (3,5), (3,8)\}$$ be the set of arcs from factories to local warehouses and $$B=\{(4,9), (5,12), (6,9), (6,10), (7,10), (7,11), (8,11), (8,12)\}$$ be the set of arcs from local warehouses to customers.

We have two sets of variables:
- Flow of units from factories to local warehouses:
$x_{ij}$, with $(i,j)\in A$
- Flow of units local warehouses to customers:
$y_{jk}$, with $(j,k)\in B$

- How would you encode the objective of this LP?

Let $c_{ij}$ denote the cost of each arc. Then, the objective is to minimize the total cost of units that are shipped trhough the network:
$$\sum_{(i,j)\in A} c_{ij}x_{ij} + \sum_{(j,k)\in B} c_{jk}y_{jk}$$

- What kind of constraints will a LP require to solve this question? First describe the effect each constraint would have in English, then write them out.

0. Firstly, we need non-negativity on the flows (no negative units):
$$x_{ij}\geq 0,\ y_{ij}\geq 0$$

1. We need a constraint that ensures that we don't exceed supply:
$$\sum_{j=4}^8 x_{ij} \leq Supply_j,\ \forall i\in\{1,2,3\}$$

2. Another constraint that enforces that we satisfy the demand:
$$\sum_{j=4}^8 y_{jk} \geq Demand_k,\forall k\in\{9,10,11,12\}$$

3. Finally, a third constraint that ensures that the amount of inventory in the warehouse is non-negative:
$$\sum_{i=1}^3 x_{ij} - \sum_{k=9}^{12} y_{jk} \geq 0,\ \forall j\in\{4,5,6,7,8\}$$

- Solve the problem in Gurobi. Make sure to save all the constraints you use (in dictionaries or as variablees), so you can later access them. 

In [8]:
# Input Data

supply = {1: 90,
          2: 40,
          3: 70}

demand = {9: 30,
          10: 50,
          11: 60,
          12: 60}

cost = {(1,4): 5, 
        (1,5): 10,
        (1,6): 11,
        (2,6): 9,
        (2,7): 9, 
        (3,5): 8,
        (3,8): 7,
        (4,9): 17,
        (5,12): 10,
        (6,9): 11,
        (6,10): 10,
        (7,10): 12,
        (7,11): 13,
        (8,11): 11,
        (8,12): 14}

A = list(cost.keys())[:7]
B = list(cost.keys())[7:]
print(A)
print(B)

[(1, 4), (1, 5), (1, 6), (2, 6), (2, 7), (3, 5), (3, 8)]
[(4, 9), (5, 12), (6, 9), (6, 10), (7, 10), (7, 11), (8, 11), (8, 12)]


In [9]:
m = gp.Model('network')

# Create variables
x = {}
for e in A:
    x[e] = m.addVar(lb=0)

y = {}
for e in B:
    y[e] = m.addVar(lb=0)

# Set objective
m.setObjective(sum(cost[e] * x[e] for e in A) + sum(cost[e] * y[e] for e in B))

# Add constraints
s = {} # Supply constraint
for i in range(1, 4): 
    s[i] = m.addConstr(sum(x[e] for e in A if e[0] == i) <= supply[i])

d = {} # Demand constraint
for k in range(9, 13): 
    d[k] = m.addConstr(sum(y[e] for e in B if e[1] == k) >= demand[k])

b = {} # Balance constraint
for j in range(4, 9): 
    b[j] = m.addConstr(sum(x[e] for e in A if e[1] == j) - sum(y[e] for e in B if e[0] == j) >= 0)
    
# Optimize
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 12 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xd1351068
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 9e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5800000e+03   1.750000e+01   0.000000e+00      0s
       7    3.8900000e+03   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds
Optimal objective  3.890000000e+03


- Your LP should have a constraint limiting the amount of supply coming out of factory 2. Query gurobi for the shadow price of that constraint. Then, rerun the optimization with that constraint somewhat relaxed, and give an intuitive economic explanation for the value of the shadow price (i.e., explain the value, in shipping costs, of increasing the capacity of factory 2), as well as how that value is created.

In [10]:
s[2].Pi

-2.0

In [11]:
# Let's add one more unit of supply to factory 2

m = gp.Model('network')

# Create variables
x = {}
for e in A:
    x[e] = m.addVar(lb=0)

y = {}
for e in B:
    y[e] = m.addVar(lb=0)

# Set objective
m.setObjective(sum(cost[e] * x[e] for e in A) + sum(cost[e] * y[e] for e in B))

# Add constraints
s = {} # Supply constraint
for i in range(1, 4): 
    s[i] = m.addConstr(sum(x[e] for e in A if e[0] == i) <= supply[i] + 1 * (i == 2))

d = {} # Demand constraint
for k in range(9, 13): 
    d[k] = m.addConstr(sum(y[e] for e in B if e[1] == k) >= demand[k])

b = {} # Balance constraint
for j in range(4, 9): 
    b[j] = m.addConstr(sum(x[e] for e in A if e[1] == j) - sum(y[e] for e in B if e[0] == j) >= 0)
    
# Optimize
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 12 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x962ac4e4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 9e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5800000e+03   1.750000e+01   0.000000e+00      0s
       7    3.8880000e+03   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds
Optimal objective  3.888000000e+03


We observe that the cost is reduced by 2, which is to say that the objective changed in $1 \cdot (-2) = 1 * \pi_2$

- Your LP should have a constraint that enforces that the amount of inventory in each warehouse is non-negative at the end of the optimization. Query the dual variable of that constraint, and explain its value.

In [12]:
for j in range(4, 9):
    print('Warehouse '+ str(j) + ': ' + str(b[j].Pi))

Warehouse 4: 5.0
Warehouse 5: 10.0
Warehouse 6: 11.0
Warehouse 7: 11.0
Warehouse 8: 9.0


These are the shadows prices associated to the balance constraints. 

If the warehouses were required to store 1 unit, these would be the extra cost.

If the warehouses could produce 1 unit, the negative of these values would be the extra savings from not havint to ship from the factory.

#### Problem 4: Facility location

In class we saw the problem of the garbage deposit stations in Amsterdam. Here, we want to solve a related problem, called ``facility location with service installation costs``. We are given, in the problem, the following:

- a set of customers that each have a service requirement
- a set of facilities that each have an opening cost (cost depends on facility) 
- a service installation cost that enables a facility to provide a particular service (depends on the facility and the service)
- an assignment cost of serving a customer through a facility (cost depends on facility and customer, but not on service)

The goal is to find (i) the set of facilities to open, (ii) what services to install in each facility, and (iii) what customers to assign to what facility so as to minimize the total costs (incl. opening, installation, and assignment).

Before you implement this in gurobi (with specific data), write down the decision variables and constraints we need to implement this optimization problem.

.

Now you should be ready to write the problem in gurobi. As usual, we first load all the data.

In [13]:
# These lists contain the facilities, the services, and the customers
facilities = ['f1', 'f2', 'f3']
services = ['s1', 's2', 's3', 's4']
customers = ['c1', 'c2', 'c3','c4','c5','c6', 'c7']

# the cost of opening any one facility are as follows
facility_opening_costs = {'f1': 13, 'f2': 15, 'f3': 12}

# Below are the costs of installing a service in an opened facility
# Notice that it's given in a so-called "nested dictionary"; the cost of installing service 
# s3 at facility f1 is service_installation_costs['f1']['s3'] which turns out 
# to be 1 (in this example)
service_installation_costs = {'f1': {'s1':2, 's2':4, 's3':1, 's4':2},
                              'f2': {'s1':1, 's2':1, 's3':1, 's4':0},
                              'f3': {'s1':2, 's2':4, 's3':1, 's4':2}}

# For each customer, the below dictionary captures what service they require
customer_service_needs = {'c1': 's1', 'c2': 's1', 'c3': 's2', 
                          'c4':'s2', 'c5':'s3', 'c6': 's3', 'c7':'s4'}

# For each pair of customer and facility, this nested dictionary captures 
# the cost of connecting facility f and customer c. so
# facility_customer_assignment_costs['f3']['c4'] would give the cost of
# connecting facility 3 and customer 4 (in this case: 6)
facility_customer_assignment_costs = {'f1': {'c1':4, 'c2':4, 'c3':3, 'c4':4, 'c5':5, 'c6':7, 'c7':8},
                                     'f2': {'c1':4, 'c2':4, 'c3':3, 'c4':4, 'c5':5, 'c6':7, 'c7':8},
                                     'f3': {'c1':5, 'c2':2, 'c3':1, 'c4':6, 'c5':15, 'c6':6, 'c7':7}
                                     }

With that data given and your IP formulation written out above, you should be able to formulate this in gurobi (optimal solution should come out as 53). If you think your formulation is correct, but you're getting stuck here, reach out to the teaching team!

To get full credit on this problem, make sure to not create variables and constraints one-by-one, i.e., the code you write should work if we were to add more facilities, services, or customers.

In [14]:
# Next, we define a gurobi optimization model
m = gp.Model("model")

# And all the decision variables

opening = {} #1 if facility f it is opened 0 if it is closed
for f in facility_opening_costs:
    opening[f] = m.addVar(name=f, vtype=gp.GRB.BINARY)

service_installations = {}
for f in service_installation_costs:
    for service in service_installation_costs[f]: #1 if the service is being installed in f 0 if not
        service_installations[f, service] = m.addVar(name=f+service, vtype=gp.GRB.BINARY)

customer_assignments = {}
for customer in customer_service_needs:
    for facility in facility_opening_costs: #1 if the customer is assigned to the facility, 0 if not
        customer_assignments[facility, customer] = m.addVar(name=customer+facility, vtype=gp.GRB.BINARY)

# We can only install a service at a facility that is opened
for facility in facilities:
    for service in services:
        m.addConstr(service_installations[facility, service] <= opening[facility])
    
# We can only assign a customer to a facility at which the required service is installeed 
for customer in customers:
    service = customer_service_needs[customer]
    for facility in facility_opening_costs:
        m.addConstr(customer_assignments[facility, customer] <= service_installations[facility, service])
#customer_assignments[facility, service] = 1 when the customer is assigned that faciltiy 0 otherwise
#so the service_installations[facility, service] must be less than or equal to the customer assignments

# Every customer needs to be assigned to a facility
for customer in customers:
    m.addConstr(sum(customer_assignments[facility, customer] for facility in facility_opening_costs) >= 1)

# We want to minimize the combined cost; it is easier to create separate expressions for each component
#OPENING COST
opening_cost = sum(facility_opening_costs[facility]*opening[facility] for facility in facility_opening_costs)


#INSTALLATION COST
#service_installations[facility, service] = 1 when it's in that facility 0 when not
installation_cost = sum(service_installations[facility, service]*service_installation_costs[facility][service]
                       for facility in facility_opening_costs for service in services)

#ASSIGNMENT COST
assignment_cost = sum(customer_assignments[facility, customer]*facility_customer_assignment_costs[facility][customer] 
                      for facility in facility_opening_costs for customer in customer_service_needs)

m.setObjective(opening_cost+installation_cost + assignment_cost, gp.GRB.MINIMIZE)

m.optimize()

for f in opening: print(f, opening[f].x)
for f,c in customer_assignments:
    if customer_assignments[f,c].x==1: print(f,c)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 40 rows, 36 columns and 87 nonzeros
Model fingerprint: 0xbff3402c
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 103.0000000
Found heuristic solution: objective 89.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 37 rows, 33 columns, 81 nonzeros
Variable types: 0 continuous, 33 integer (33 binary)

Root relaxation: objective 5.300000e+01, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      53.0000000   53.00000  0.00%     -    0s

Explored 0 nodes (26 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available pro

Suppose we can only connect at most 4 customers to a facility. Take your previous optimization problem and solve it with such a constraint added for each facility (optimal solution should come out as 65).

In [15]:
# First, we definee a gurobi optimization model
m = gp.Model("model")

# And all the decision variables

opening = {} #1 if facility f it is opened 0 if it is closed
for f in facility_opening_costs:
    opening[f] = m.addVar(name=f, vtype=gp.GRB.BINARY)

service_installations = {}
for f in service_installation_costs:
    for service in service_installation_costs[f]: #1 if the service is being installed in f 0 if not
        service_installations[f, service] = m.addVar(name=f+service, vtype=gp.GRB.BINARY)

customer_assignments = {}
for customer in customer_service_needs:
    for facility in facility_opening_costs: #1 if the customer is assigned to the facility, 0 if not
        customer_assignments[facility, customer] = m.addVar(name=customer+facility, vtype=gp.GRB.BINARY)

# We can only install a service at a facility that is opened
for facility in facilities:
    # HERE IS THE NEW CONSTRAINT THAT CAPACITATES NUMBER OF CUSTOMERS FOR A FACILITY
    m.addConstr(sum(customer_assignments[facility, customer] for customer in customers)<=4)
    for service in services:
        m.addConstr(service_installations[facility, service] <= opening[facility])
    
# We can only assign a customer to a facility at which the required service is installeed 
for customer in customers:
    service = customer_service_needs[customer]
    for facility in facility_opening_costs:
        m.addConstr(customer_assignments[facility, customer] <= service_installations[facility, service])
        
#customer_assignments[facility, service] = 1 when the customer is assigned that faciltiy 
#0 otherwise
#so the service_installations[facility, service] must be less than or equal to the customer assignments

# Every customer needs to be assigned to a facility
for customer in customers:
    m.addConstr(sum(customer_assignments[facility, customer] for facility in facility_opening_costs) >= 1)

# We want to minimize the combined cost; it is easier to create separate expressions for each component
#OPENING COST
opening_cost = sum(facility_opening_costs[facility]*opening[facility] for facility in facility_opening_costs)


#INSTALLATION COST
#service_installations[facility, service] = 1 when it's in that facility 0 when not

installation_cost = sum(service_installations[facility, service]*service_installation_costs[facility][service]
                       for facility in facility_opening_costs for service in services)

#ASSIGNMENT COST
assignment_cost = sum(customer_assignments[facility, customer]*facility_customer_assignment_costs[facility][customer] 
                      for facility in facility_opening_costs for customer in customer_service_needs)

m.setObjective(opening_cost+installation_cost + assignment_cost, gp.GRB.MINIMIZE)

m.optimize()

for f in opening: print(f, opening[f].x)
for f,c in customer_assignments:
    if customer_assignments[f,c].x==1: print(f,c)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 43 rows, 36 columns and 108 nonzeros
Model fingerprint: 0x01839b2e
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 89.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 40 rows, 33 columns, 102 nonzeros
Variable types: 0 continuous, 33 integer (33 binary)

Root relaxation: objective 5.471429e+01, 35 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   54.71429    0   22   89.00000   54.71429  38.5%     -    0s
H    0     0                      80.0000000   54.71429  31.6%     -    0s
H    0     0                      66.0000000   54.71429  17.1%     